In [ ]:
!pip install langchain langchain_openai openai langchain-pinecone pinecone-notebooks langchain-community tavily-python smolagents --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1

In [ ]:
import pandas as pd
import os
import json
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langsmith import traceable
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from smolagents import CodeAgent, ToolCallingAgent, ManagedAgent, DuckDuckGoSearchTool, VisitWebpageTool

ImportError: cannot import name 'ManagedAgent' from 'smolagents' (/usr/local/lib/python3.11/dist-packages/smolagents/__init__.py)

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Set display width
pd.set_option('display.max_colwidth', None)  # Allow long text in a single cell

**Setting API KEYS and Importing Data**

In [ ]:
os.environ["PINECONE_API_KEY"]="API KEY"
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "API KEY"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
demographics = pd.read_parquet('/content/drive/MyDrive/Data/demographics.parquet')
diagnoses = pd.read_parquet('/content/drive/MyDrive/Data/diagnoses.parquet')
medications = pd.read_parquet('/content/drive/MyDrive/Data/medications.parquet')
notes = pd.read_parquet('/content/drive/MyDrive/Data/notes.parquet')
orders = pd.read_parquet('/content/drive/MyDrive/Data/orders.parquet')
vitals = pd.read_parquet('/content/drive/MyDrive/Data/vitals.parquet')
ordersclassified = pd.read_csv('/content/drive/MyDrive/Data/cleaned_orders.csv')

**Initializing Vector Store**

In [ ]:
pc = Pinecone()

In [ ]:
embeddings=OpenAIEmbeddings(openai_api_key="API KEY")

In [ ]:
index = pc.Index("estimating-ehr-costs")
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
index_stats = index.describe_index_stats()
index_stats

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 83555}},
 'total_vector_count': 83555}

**Filtering for CT Chest Scan Orders >= 18 years Old**

In [ ]:
ct_chest_orders = ordersclassified[ordersclassified["Ed's Term"]=='CT chest']
ct_orders_dem = pd.merge(ct_chest_orders, demographics, on="VisitKey_hashed", how="left")
acep22orders = ct_orders_dem[(ct_orders_dem['age_group'].apply(lambda x: int(x.split('-')[0].replace('+', '')) >= 20))]


In [ ]:
# Drop duplicates based on 'VisitKey_hashed' and keep the first occurrence
acep22_visits = acep22orders[['VisitKey_hashed', 'LocationID_hashed']].drop_duplicates(subset='VisitKey_hashed', keep='first')

In [ ]:
set(demographics['LocationID_hashed']) == set(notes['FacilityCode_hashed'])

True

In [ ]:
acep22_visits.groupby('LocationID_hashed').count().sort_values(by='VisitKey_hashed', ascending=False)

,VisitKey_hashed
LocationID_hashed,
2b4da2f6b7d30484b4962a682c5390aa823537fb78176a1ba55c59b2af2ba99b,158
9ac202a84e32bf103066d58172339c2dd76980af8412da04d2de0a6adf91c654,128
9eeabd9a714b2a462e6fd844ef0ab3dd95e3f3f8734cdd2f17cfbdee936fc9a1,75
aeb2ef0493ab863f1fd0c64bc5cc74b1447517e8aa10590001870d4e4632e585,72
3db45dff765ac59f43dfad9730315613930c16720c694c59bb44d5e733a15577,71
...,...
55c85b838d6dd4d4ae090a3a01397b178d503621e2e36125a60adba09c4701ca,1
a2c3ce79071d865fc225352f416f97c2470ba60310a1801cf61bb7003049314f,1
d9b38f038fb2f27e927b8be0a754b237b703fae561cb264ec0c0a5ada8788cf8,1


**Filtering on a Specific Medical Center**

In [ ]:
visits_for_rag = acep22_visits[acep22_visits.LocationID_hashed == '2b4da2f6b7d30484b4962a682c5390aa823537fb78176a1ba55c59b2af2ba99b'][0:25]

**Initializing LLM**

In [ ]:
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",  # You can switch to other models supported by OpenRouter
    temperature=0,
    openai_api_key="API KEY",
    openai_api_base="https://openrouter.ai/api/v1" # Specify the base URL for OpenRouter
)

In [ ]:
import requests

# Replace <OPENROUTER_API_KEY> with your actual API key
OPENROUTER_API_KEY = "API KEY"

# Define the API endpoint and headers
url = "https://openrouter.ai/api/v1/auth/key"
headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}"
}

try:
    # Send the GET request
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the JSON response
    data = response.json()
    print("Label:", data["data"]["label"])
    print("Usage:", data["data"]["usage"])  # Number of credits used
    print("Limit:", data["data"]["limit"])  # Credit limit for the key, or None if unlimited
    print("Is Free Tier:", data["data"]["is_free_tier"])
    print("Rate Limit:", data["data"]["rate_limit"])

except requests.exceptions.RequestException as e:
    print("Error:", e)


Label: sk-or-v1-7c5...04d
Usage: 0.3511502055
Limit: None
Is Free Tier: False
Rate Limit: {'requests': 840, 'interval': '10s'}


**Creating Agent that Should Take A Question and Look up More information on it On MD Calc to Support Answer**

In [ ]:
os.environ["TAVILY_API_KEY"] = "API KEY"
search_tool = TavilySearchResults(
  name="tavily_search_engine",
  description="A search engine optimized for retrieving information from web based on user query.",
  max_results=4,
  search_depth="advanced",
  include_answer=True,
  include_raw_content=True,
  include_images=True,
  verbose=False,
)

In [ ]:
from langchain_core.prompts import (
  ChatPromptTemplate,
)
prompt = ChatPromptTemplate([
  ("system",
  f"""You are a medical chatbot that aims to help physicians diagnose their medical notes. You need to answer the user's queries in
  detail from the document context. You have access to two tools:
  pinecone_vectorstore_retriever and tavily_search_engine.
  Before extracting the information from the pinecone_vectorstore_retriever, use the tavily_search_engine to open mdcalc.com and search for the necessary information to answer the question.
  If the tavily_search_engine is used, specify what website was used to extract this information.
  Only look up information when you are sure of what you want.
  If you need to look up some information before asking a follow up question, you are allowed to do that.
  Always use the pinecone_vectorstore_retriever tool first to retrieve the context and answer the question. NEVER give an incomplete answer.
  When using the pinecone_vectorstore_retriever tool, specify where the information is extracted from.
  Always try your best to find answer through web search if answer is not found from context."""),
  ("human", "{user_input}"),
  ("placeholder", "{messages}"),
  ("placeholder", "{agent_scratchpad}"),
])

In [ ]:
from langchain.tools.retriever import create_retriever_tool

filters = {"VisitID": "5fe2b1b5292f0553873da5bfc6ea48a5ab7e31572b4df98cfa7226c278151a31"}  # Add any metadata filters here


retriever_tool = create_retriever_tool(
	retriever=vector_store.as_retriever(
	search_type="similarity",
	search_kwargs={
  "filter": filters
	},
 return_source_documents=True
),
name="pinecone_vectorstore_retriever",
  description="Searches and returns contents from uploaded medical document based on user query.",
)

tools = [
  retriever_tool,
  search_tool,
]

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
  tools=tools,
  agent=agent,
  verbose=False,
  max_iterations=8,
  early_stopping_method="force"
)

In [ ]:
result_q2_background = agent_executor.invoke({
  "user_input": """

           Use https://www.mdcalc.com/calc/115/wells-criteria-pulmonary-embolism#when-to-use to determine how to calculate moderate or high pre-test clinical probablity for pulmonary embolism

           """,
},)
print(result_q2_background["output"])

To calculate moderate or high pre-test clinical probability for pulmonary embolism (PE) using the Wells criteria, the following points are considered:

1. **Clinical Features**: The Wells criteria assigns points based on specific clinical features. The total score helps categorize the probability of PE:
   - **High Probability**: A score of 6 or more.
   - **Moderate Probability**: A score of 2 to 6.
   - **Low Probability**: A score of less than 2.

2. **Criteria for Scoring**:
   - Clinical signs and symptoms of deep vein thrombosis (DVT) (3 points)
   - An alternative diagnosis less likely than PE (3 points)
   - Heart rate greater than 100 beats per minute (1.5 points)
   - Immobilization for more than 3 days or surgery in the previous 4 weeks (1.5 points)
   - Previous DVT or PE (1.5 points)
   - Hemoptysis (1 point)
   - Malignancy (1 point)

3. **Application**: The Wells criteria should be applied after a thorough history and physical examination suggests that venous thromboembo

**Using Information from Agent in Prompt for RAG to Answer if Visit Had Moderate to High Probability of PE**

In [ ]:
system_prompt = (
    """
    You are a medical expert specializing in emergency healthcare visits. Your task is to assess whether a patient has a **moderate or high pre-test clinical probability for pulmonary embolism (PE)** based on the **Wells criteria**.

### **Instructions:**
- Carefully analyze the **context** provided.
- Evaluate each criterion **individually** and determine whether it applies.
- Clearly state **"Yes" or "No"**, followed by a **brief, evidence-based reasoning** for your conclusion.

### **Wells Criteria for Pre-Test Probability of Pulmonary Embolism (PE):**
- **High Probability**: Score ≥6
- **Moderate Probability**: Score 2 to 6
- **Low Probability**: Score <2

#### **Scoring System:**
1. Clinical signs and symptoms of deep vein thrombosis (DVT) = 3 points
2. Pulmonary embolism (PE) is the #1 diagnosis OR equally likely = 3 points
3. Heart rate >100 beats per minute = 1.5 points
4. Immobilization for >3 days or recent surgery (past 4 weeks) = 1.5 points
5. History of DVT or PE = 1.5 points
6. Hemoptysis (coughing up blood) = 1 point
7. Malignancy (active cancer or receiving treatment) = 1 point

### **Output Format:**
- Clearly state "Yes" or "No", followed by a concise explanation.
- If "Yes," provide a breakdown of applicable criteria and total score.
- If "No," explain why the patient does not meet the threshold.

### **Example Output:
Yes. The patient has a swollen leg which is a clinical sign of DVT (3 points), a heart rate of 110 (1.5 points), and a history of PE (1.5 points), leading to a total score of 6, which indicates high probability of PE.

OR

No. The patient has a Wells score of 1.5 due to immobilization, which falls below the **moderate probability threshold.

Use the provided context to justify your answer.

{context}

    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


filters = {"VisitID": "770ea9e81984e345ec06a03a5cafae35ce3798b31c076a2781b3474a2b71b591"}
retriever=vector_store.as_retriever(search_kwargs={"filter": filters})
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Does this visit indicate the patient had a moderate or high probability for pulmonary embolism BEFORE any tests were run?"})
response["answer"]

"No. The patient does not meet the criteria for moderate or high probability for pulmonary embolism (PE) based on the Wells criteria. \n\n### Breakdown of Criteria:\n1. **Clinical signs and symptoms of deep vein thrombosis (DVT)**: No evidence of DVT was mentioned.\n2. **Pulmonary embolism (PE) is the #1 diagnosis OR equally likely**: The differential diagnosis includes PE but does not indicate it as the primary diagnosis.\n3. **Heart rate >100 beats per minute**: The patient's pulse was recorded at 69-75 beats per minute, which does not meet this criterion.\n4. **Immobilization for >3 days or recent surgery (past 4 weeks)**: No mention of recent immobilization or surgery.\n5. **History of DVT or PE**: The patient has a history of PE, which would score 1.5 points.\n6. **Hemoptysis (coughing up blood)**: No hemoptysis was reported.\n7. **Malignancy (active cancer or receiving treatment)**: No mention of malignancy.\n\n### Total Score:\n- History of DVT or PE: 1.5 points\n\nThe total sco

**Looping Through Filtered Visits**

In [ ]:
visit_ids = list(visits_for_rag['VisitKey_hashed'])
# Initialize a results list
results = []

# Loop over each VisitID
for visit_id in visit_ids:
    # Dynamically update the filter
    filters = {"VisitID": visit_id}

    # Create retriever with updated filter
    retriever = vector_store.as_retriever(search_kwargs={"filter": filters})

    # Create and run the RAG chain
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Invoke the chain with the input question
    response = rag_chain.invoke({"input": "Does this visit indicate the patient had moderate or high clinical probability for pulmonary embolism BEFORE any tests were run?"})

    # Store the result
    results.append({"VisitID": visit_id, "Answer 1": response["answer"]})

In [ ]:
question_1_results = pd.DataFrame(results)

**Using RAG to Answer if Visit Had Elevated D-Dimer levels**

In [ ]:
system_prompt = (
    """
    You are a highly skilled medical expert specializing in emergency healthcare.
    Your task is to analyze medical notes from emergency health visits and provide precise, evidence-based answers.

    Instructions:
    - Carefully analyze the context and question before responding.
    - Answer in the following format:
      - Start with "Yes." or "No."
      - Provide a clear, concise medical explanation for your answer.
      - If applicable, cite relevant clinical findings or reasoning from the provided context.

    Context:
    {context}

    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


filters = {"VisitID": "5fe2b1b5292f0553873da5bfc6ea48a5ab7e31572b4df98cfa7226c278151a31"}
retriever=vector_store.as_retriever(search_kwargs={"filter": filters})
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Does this visit indicate the patient had elevated d-dimer levels?"})
response["answer"]

"No. \n\nThe provided medical notes do not mention any d-dimer levels or tests performed related to d-dimer. The focus of the notes is on the patient's chest pain, past medical history, and results from imaging and lab tests, but there is no indication of a d-dimer test being conducted or its results. Elevated d-dimer levels are typically associated with conditions such as pulmonary embolism or deep vein thrombosis, which are not indicated in this patient's evaluation."

**Looping Through Filtered Visits**

In [ ]:
visit_ids = list(visits_for_rag['VisitKey_hashed'])
# Initialize a results list
q2_results = []

# Loop over each VisitID
for visit_id in visit_ids:
    # Dynamically update the filter
    filters = {"VisitID": visit_id}

    # Create retriever with updated filter
    retriever = vector_store.as_retriever(search_kwargs={"filter": filters})

    # Create and run the RAG chain
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Invoke the chain with the input question
    response = rag_chain.invoke({"input": "Does this visit indicate the patient had elevated d-dimer levels?"})

    # Store the result
    q2_results.append({"VisitID": visit_id, "Answer 2": response["answer"]})

In [ ]:
question_2_results = pd.DataFrame(q2_results)

In [ ]:
question1_2 = question_1_results.merge(question_2_results, on="VisitID", how="inner")

In [ ]:
# Filter rows where both "Answer 1" and "Answer 2" start with "No"
question1_2_filtered = question1_2[
    question1_2["Answer 1"].str.startswith("No") & question1_2["Answer 2"].str.startswith("No")
]

**Using the Agent to Provide Background on Next Question**

In [ ]:
result_q3_background = agent_executor.invoke({
  "user_input": """

           Use md.calc to determine some medical reasons for ordering a CTPA WITHOUT moderate or high pre-test clinical probability for pulmonary embolism or no positive result of elevated D-dimer level (e.g., CT ordered for aortic dissection)

           """,
},)

To calculate moderate or high pre-test clinical probability for pulmonary embolism (PE) using the Wells criteria, the following points are considered:

1. **Clinical Features**: The Wells criteria assigns points based on specific clinical features. The total score helps categorize the probability of PE:
   - **High Probability**: A score of 6 or more.
   - **Moderate Probability**: A score of 2 to 6.
   - **Low Probability**: A score of less than 2.

2. **Criteria for Scoring**:
   - Clinical signs and symptoms of deep vein thrombosis (DVT) (3 points)
   - An alternative diagnosis less likely than PE (3 points)
   - Heart rate greater than 100 beats per minute (1.5 points)
   - Immobilization for more than 3 days or surgery in the previous 4 weeks (1.5 points)
   - Previous DVT or PE (1.5 points)
   - Hemoptysis (1 point)
   - Malignancy (1 point)

3. **Application**: The Wells criteria should be applied after a thorough history and physical examination suggests that venous thromboembo

In [ ]:
result_q3_background["output"]

'Computed Tomography Pulmonary Angiography (CTPA) can be ordered for various medical reasons even when there is no moderate or high pre-test clinical probability for pulmonary embolism (PE) or when D-dimer levels are not elevated. Here are some indications based on the context and additional information:\n\n1. **Aortic Dissection**: CTPA is often utilized to evaluate suspected aortic dissection. In cases where a patient presents with chest pain and "kidney pain," as noted in the medical history, aortic dissection becomes a differential diagnosis. The imaging can help rule out or confirm the presence of an aortic dissection.\n\n2. **Other Vascular Conditions**: CTPA can be used to assess other vascular conditions, such as pulmonary artery anomalies or other thoracic vascular pathologies, which may not be directly related to PE.\n\n3. **Chest Pain Evaluation**: In patients presenting with chest pain, CTPA can be part of a broader diagnostic workup to exclude other serious conditions, suc

**Using the info from the agent to answer if the visit has another medical reason for ordering a CTPA scan**

In [ ]:
system_prompt = (
    """
    You are a medical expert specializing in emergency healthcare.
    Your task is to analyze medical notes from emergency health visits and provide precise, evidence-based answers.

    Instructions:
    - Carefully analyze the context and question before responding.
    - Answer in the following format:
      - Start with "Yes." or "No."
      - Provide a clear, concise medical explanation for your answer.

    Medical Background:
    Computed Tomography Pulmonary Angiography (CTPA) can be ordered for various medical reasons even when there is no moderate or high pre-test clinical probability for pulmonary embolism (PE) or when D-dimer levels are not elevated.
    Here are some indications based on the context from MdCalc:
    **Aortic Dissection**: CTPA is often utilized to evaluate suspected aortic dissection.
    In cases where a patient presents with chest pain and kidney pain, aortic dissection becomes a differential diagnosis.
    The imaging can help rule out or confirm the presence of an aortic dissection.
    **Other Vascular Conditions**: CTPA can be used to assess other vascular conditions, such as pulmonary artery anomalies or other thoracic vascular pathologies, which may not be directly related to PE.
    **Chest Pain Evaluation**: In patients presenting with chest pain, CTPA can be part of a broader diagnostic workup to exclude other serious conditions, such as aortic aneurysm or other thoracic masses, even if the likelihood of PE is low.
    **Negative Findings in Other Tests**: If initial tests (like EKG or chest X-ray) are inconclusive or negative, and the clinical suspicion for other conditions remains high, CTPA may be warranted to provide further clarity.

    Context:
    {context}

    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


filters = {"VisitID": "d7723343f73d5c6b8fcdd8721156e4f26b6021b2485899ca5bc8a12554cf4dce"}
retriever=vector_store.as_retriever(search_kwargs={"filter": filters})
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Does this visit indicate there was a medical reason for ordering a CTPA WITHOUT having a moderate or high pre-test clinical probability for pulmonary embolism or a positive result of elevated D-dimer level?"})
response["answer"]

"Yes. The visit indicates a medical reason for ordering a CTPA without having a moderate or high pre-test clinical probability for pulmonary embolism or a positive result of elevated D-dimer levels. The presence of right-sided facial bone fractures and a tiny pneumothorax, along with the patient's trauma history, suggests the need to evaluate for other serious conditions, such as aortic dissection or other vascular injuries, which can be assessed using CTPA. This imaging can help rule out or confirm these conditions even in the absence of elevated D-dimer levels or high clinical suspicion for PE."

**Looping Through Filtered Visits**

In [ ]:
visit_ids = list(question1_2_filtered['VisitID'])
# Initialize a results list
q3_results = []

# Loop over each VisitID
for visit_id in visit_ids:
    # Dynamically update the filter
    filters = {"VisitID": visit_id}

    # Create retriever with updated filter
    retriever = vector_store.as_retriever(search_kwargs={"filter": filters})

    # Create and run the RAG chain
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Invoke the chain with the input question
    response = rag_chain.invoke({"input": "Does this visit indicate there was a medical reason for ordering a CTPA WITHOUT having a moderate or high pre-test clinical probability for pulmonary embolism or a positive result of elevated D-dimer level?"})

    # Store the result
    q3_results.append({"VisitID": visit_id, "Answer 3": response["answer"]})

In [ ]:
question_3_results = pd.DataFrame(q3_results)

**Using RAG To see if the CT Scan was ordered due to trauma or injury**

In [ ]:
system_prompt = (
    """
    You are a highly skilled medical expert specializing in emergency healthcare.
    Your task is to analyze medical notes from emergency health visits and provide precise, evidence-based answers.

    Instructions:
    - Carefully analyze the context and question before responding.
    - Answer in the following format:
      - Start with "Yes." or "No."
      - Provide a clear, concise medical explanation for your answer.
      - If applicable, cite relevant clinical findings or reasoning from the provided context.

    Context:
    {context}

    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


filters = {"VisitID": "d7723343f73d5c6b8fcdd8721156e4f26b6021b2485899ca5bc8a12554cf4dce"}
retriever=vector_store.as_retriever(search_kwargs={"filter": filters})
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Does this visit indicate the patient had the CTPA ordered for trauma or dangerous mechanism of injury?"})
response["answer"]

"Yes. The visit indicates that a CTA (CT Angiography) of the head and neck was ordered, which is typically performed in cases of trauma or dangerous mechanisms of injury to assess for vascular injuries, such as major vessel occlusions or significant carotid stenosis. The patient's presentation of thoracic and intra-abdominal injuries, along with facial bone fractures and a history of being a pedestrian injured in traffic, supports the need for such imaging to evaluate potential vascular compromise."

**Looping Through Visits**

In [ ]:
visit_ids = list(visits_for_rag['VisitKey_hashed'])
# Initialize a results list
q4_results = []

# Loop over each VisitID
for visit_id in visit_ids:
    # Dynamically update the filter
    filters = {"VisitID": visit_id}

    # Create retriever with updated filter
    retriever = vector_store.as_retriever(search_kwargs={"filter": filters})

    # Create and run the RAG chain
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Invoke the chain with the input question
    response = rag_chain.invoke({"input": "Does this visit indicate the patient had the CTPA ordered for trauma or dangerous mechanism of injury?"})

    # Store the result
    q4_results.append({"VisitID": visit_id, "Answer 4": response["answer"]})

In [ ]:
question_4_results = pd.DataFrame(q4_results)

**Using RAG to see if the patient was pregnant**

In [ ]:
system_prompt = (
    """
    You are a highly skilled medical expert specializing in emergency healthcare.
    Your task is to analyze medical notes from emergency health visits and provide precise, evidence-based answers.

    Instructions:
    - Carefully analyze the context and question before responding.
    - Answer in the following format:
      - Start with "Yes." or "No."
      - Provide a clear, concise medical explanation for your answer.
      - If applicable, cite relevant clinical findings or reasoning from the provided context.

    Context:
    {context}

    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


filters = {"VisitID": "28fcf4acc3d147ae1d38826ee98bcfd1738a1aab5e7ef93151d50c755f254695"}
retriever=vector_store.as_retriever(search_kwargs={"filter": filters})
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Does this visit indicate the patient was pregnant?"})
response["answer"]

"No. The provided medical notes do not indicate any information regarding the patient's pregnancy status. There are no mentions of pregnancy-related symptoms, history, or tests that would suggest the patient is pregnant."

**Looping through the Visits**

In [ ]:
visit_ids = list(visits_for_rag['VisitKey_hashed'])
# Initialize a results list
q5_results = []

# Loop over each VisitID
for visit_id in visit_ids:
    # Dynamically update the filter
    filters = {"VisitID": visit_id}

    # Create retriever with updated filter
    retriever = vector_store.as_retriever(search_kwargs={"filter": filters})

    # Create and run the RAG chain
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    # Invoke the chain with the input question
    response = rag_chain.invoke({"input": "Does this visit indicate the patient was pregnant?"})

    # Store the result
    q5_results.append({"VisitID": visit_id, "Answer 5": response["answer"]})

In [ ]:
question_5_results = pd.DataFrame(q5_results)

**Merging all The Answers and Assigning Points to Responses 'Yes = 1, No = 0'**

In [ ]:
acep_22_answers = question_1_results.merge(question_2_results, on="VisitID", how="inner")\
               .merge(question_3_results, on="VisitID", how="left")\
               .merge(question_4_results, on="VisitID", how="inner")\
               .merge(question_5_results, on="VisitID", how="inner")

In [ ]:
columns_to_clean = ["Answer 1", "Answer 2", "Answer 3", "Answer 4", "Answer 5"]

# Remove '\n' from specified columns
acep_22_answers[columns_to_clean] = acep_22_answers[columns_to_clean].apply(lambda x: x.str.replace("\n", " ", regex=True))

In [ ]:
acep_22_answers = acep_22_answers.merge(notes, left_on="VisitID", right_on="USACSKEY_hashed", how="inner")


In [ ]:
acep_22_answers = acep_22_answers[['VisitID','NoteTextCombined','Answer 1','Answer 2','Answer 3','Answer 4','Answer 5']]

In [ ]:
acep_22_answers.to_csv("/content/drive/MyDrive/Data/acep_22_answers.csv")

In [ ]:
# Define the columns to check
columns_to_check = ["Answer 3", "Answer 4", "Answer 5"]

# Create the new column with the fixed logic
acep_22_answers["Denominator"] = acep_22_answers[columns_to_check].apply(
    lambda row: 0 if any(str(row[col]).startswith("Yes") for col in columns_to_check) else 1, axis=1
)

In [ ]:
# Define the columns to check
columns_to_check = ["Answer 1", "Answer 2"]

# Create the new column with the fixed logic
acep_22_answers["Numerator"] = acep_22_answers[columns_to_check].apply(
    lambda row: 1 if any(str(row[col]).startswith("Yes") for col in columns_to_check) else 0, axis=1
)

In [ ]:
acep_22_answers.to_csv("/content/drive/MyDrive/Data/acep_22_answers.csv")